# Comprehensive Plotting Example for Refactored PV Plotting Library

This notebook demonstrates the plotting capabilities of the refactored pvplotting library using real RAMS simulation data and trajectory data from the isolated convection case study. It serves as both a comprehensive test of the refactored system and a practical example for scientific visualization.

## Data Source
- RAMS simulation: Isolated convection case with warm bubble forcing
- Trajectory data: Forward and backward trajectories through the convective system
- Time range: 1991-04-26 21:00:00 to 1991-04-27 00:00:00 (3 hours)
- Spatial resolution: 500m horizontal, variable vertical

## Setup and Imports

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import datetime as dt
from pathlib import Path
import time
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Image
import warnings

# warnings.filterwarnings('ignore')

# Import the refactored plotting library
import common_experimental as cm
from common_experimental.pvplotting.types_pvplotting import (
    PVConfig,
    PVRamsData,
    PVTrajectoryData,
    PVContourSpec,
    PV2DSpec,
    PVVectorSpec,
    PVTrajectorySpec,
)
from common_experimental.pvplotting.core_pvplotting import plot_rams_and_trajectories
from common_experimental.pvplotting.plotter import initialize_plotter

# Set up PyVista for notebook rendering
import pyvista as pv

print("✓ All imports successful")
print(f"PyVista version: {pv.__version__}")
print(f"Backend: {pv.global_theme.jupyter_backend}")

Not working within blender, so not importing
✓ All imports successful
PyVista version: 0.45.3
Backend: trame


## Data Loading and Preparation

Load the same RAMS and trajectory data used in the "Paper" section of the analysis notebook.

In [2]:
# Data paths (matching the Paper section configuration)
storm_dir = Path(
    "/moonbow/cmdavis4/projects/bl_transport/rams_io/isolated_convection_wk/all_tracers/newtracer_forcing-warmbubble4_deltax-500_frqlite-2_nobulku"
)
trajectories_dir = storm_dir.joinpath("trajectories")
rams_output_dir = storm_dir.joinpath("output")

# Time configuration
start_time = pd.Timestamp("1991-04-26 21:00:00").to_pydatetime()
end_time = pd.Timestamp("1991-04-27 00:00:00").to_pydatetime()
timestep = dt.timedelta(minutes=5)
# Match the datetimes we'll read in based on what's in the trajectories
included_dts = pd.date_range(
    start=start_time,
    end=end_time,
    freq=timestep,
)

print(f"Data directory: {storm_dir}")
print(f"Directory exists: {storm_dir.exists()}")
print(f"Time range: {start_time} to {end_time}")
print(f"Timestep: {timestep}")

Data directory: /moonbow/cmdavis4/projects/bl_transport/rams_io/isolated_convection_wk/all_tracers/newtracer_forcing-warmbubble4_deltax-500_frqlite-2_nobulku
Directory exists: True
Time range: 1991-04-26 21:00:00 to 1991-04-27 00:00:00
Timestep: 0:05:00


In [3]:
# Load trajectory data (use preprocessed if available)
preprocessed_trajectory_file = trajectories_dir / "ft_bt_paper.nc"

if preprocessed_trajectory_file.exists():
    print("Loading preprocessed trajectory data...")
    trajectory_ds = xr.open_dataset(preprocessed_trajectory_file)
    print(
        f"Loaded {len(trajectory_ds.parcel_ix)} parcels over"
        f" {len(trajectory_ds.time)} time steps"
    )
else:
    print(
        "Preprocessed file not found. This example requires the preprocessed trajectory"
        " data."
    )
    print(
        "Please run the Paper section of analysis_storm-ic.ipynb first to generate the"
        " data."
    )
    raise FileNotFoundError(f"Required file not found: {preprocessed_trajectory_file}")

# Limit to values from times
# Display basic trajectory dataset info
print("\nTrajectory dataset structure:")
print(f"Dimensions: {dict(trajectory_ds.dims)}")
print(f"Key variables: {list(trajectory_ds.data_vars)[:10]}...")  # Show first 10

Loading preprocessed trajectory data...
Loaded 257053 parcels over 1081 time steps

Trajectory dataset structure:
Dimensions: {'parcel_ix': 257053, 'time': 1081}
Key variables: ['x_ix', 'x', 'y_ix', 'y', 'z_ix', 'z', 'converged', 'n_iterations', 'oob', 'UC']...


In [4]:
trajectory_input_fnames = [
    rams_output_dir.joinpath(f"a-L-{x}-g1.h5")
    for x in included_dts.strftime(cm.rams.RAMS_DT_STRFTIME_STR).values
]
assert all([x.exists() for x in trajectory_input_fnames])

# Read in the data
rams_ds = cm.rams.read_rams_output(
    input_filenames=trajectory_input_fnames,
    time_dim_name="time",
    # keep_vars=["UC", "VC", "WC"] + tracked_scalars,
    dim_names={
        "phony_dim_0": "y",
        "phony_dim_1": "x",
        "phony_dim_2": "p",
        "phony_dim_3": "z",
    },
)

Reading and concatenating 37 RAMS individual timestep outputs...
[########################################] | 100% Completed | 4.77 ss


## Convert to New Data Classes

Convert the loaded data to the new `PVRamsData` and `PVTrajectoryData` classes with appropriate variable specifications.

In [5]:
# Create trajectory variable specifications for different visualization types
trajectory_specs = [
    # Basic trajectory lines colored by height
    PVTrajectorySpec(
        varname="trajectories_by_height",
        scalar="z",
        cmap="terrain",
        scalar_bar=True,
        scalar_bar_args={"title": "Height (m)", "position_x": 0.85},
    ),
    # Trajectories colored by group (if available)
    (
        PVTrajectorySpec(
            varname="trajectories_by_group",
            scalar="group_ix" if "group_ix" in trajectory_ds.data_vars else "z",
            cmap="tab10",
            scalar_bar=True,
            scalar_bar_args={"title": "Group", "position_x": 0.15},
        )
        if "group_ix" in trajectory_ds.data_vars
        else None
    ),
    # Particle-style trajectories
    PVTrajectorySpec(
        varname="trajectory_particles",
        particles=True,
        scalar="z",
        cmap="viridis",
        body_radius=50,
    ),
]

# Filter out None specs
trajectory_specs = [spec for spec in trajectory_specs if spec is not None]

print(f"Created {len(trajectory_specs)} trajectory specifications")
for spec in trajectory_specs:
    print(f"  - {spec.varname}: scalar={spec.scalar}, particles={spec.particles}")

Created 3 trajectory specifications
  - trajectories_by_height: scalar=z, particles=False
  - trajectories_by_group: scalar=group_ix, particles=False
  - trajectory_particles: scalar=z, particles=True


In [6]:
# Create PVTrajectoryData object
print("Creating PVTrajectoryData object...")

# Limit the number of parcels for performance (use first 200 parcels)
limited_trajectory_ds = trajectory_ds.isel(parcel_ix=slice(0, 200))

trajectory_data = PVTrajectoryData(
    trajectory_ds=limited_trajectory_ds,
    varspecs=tuple(trajectory_specs),
    n_parcel_limit=200,  # Explicitly set limit
)

print(
    f"✓ Created trajectory data object with {len(trajectory_data.ds.parcel_ix)} parcels"
)
print(
    f"  Time range: {trajectory_data.ds.time.values[0]} to"
    f" {trajectory_data.ds.time.values[-1]}"
)
print(f"  Variables: {len(trajectory_specs)} trajectory visualizations")

Creating PVTrajectoryData object...
✓ Created trajectory data object with 200 parcels
  Time range: 1991-04-26T21:00:00.000000000 to 1991-04-27T00:00:00.000000000
  Variables: 3 trajectory visualizations


In [7]:
# Create RAMS data object (if available)
rams_data = None
if rams_ds is not None:
    print("Creating PVRamsData object...")

    # Example RAMS variable specifications
    rams_specs = [
        # Temperature contours
        PVContourSpec(
            varname="THETA",
            isosurfaces=[300, 305, 310, 315],
            scalar_bar=True,
            add_mesh_kwargs={"opacity": 0.6, "cmap": "coolwarm"},
        ),
        # Humidity contours
        # PVContourSpec(
        #     varname="RV",
        #     isosurfaces=[0.01, 0.015, 0.02],
        #     scalar_bar=True,
        #     add_mesh_kwargs={"opacity": 0.4, "cmap": "Blues"},
        # ),
        # # Wind vectors
        # PVVectorSpec(
        #     varname="wind",
        #     u_varname="UC",
        #     v_varname="VC",
        #     w_varname="WC",
        #     add_mesh_kwargs={"factor": 100, "opacity": 0.7},
        # ),
    ]

    rams_data = PVRamsData(simulation_ds=rams_ds, varspecs=tuple(rams_specs))
    print(f"✓ Created RAMS data object with {len(rams_specs)} variables")
else:
    print("Skipping RAMS data object creation - focusing on trajectories only")

Creating PVRamsData object...
✓ Created RAMS data object with 1 variables


## Performance Monitoring Setup

In [8]:
# Performance monitoring utilities
def time_operation(operation_name, func, *args, profile=False, **kwargs):
    """Time an operation and return results with timing info.

    Args:
        operation_name: Name of the operation for logging
        func: Function to execute
        *args: Arguments to pass to func
        profile: If True, run cProfile to identify bottlenecks
        **kwargs: Keyword arguments to pass to func

    Returns:
        tuple: (result, duration) or (result, duration, profile_stats) if profile=True
    """
    print(f"Starting {operation_name}...")
    start_time = time.time()

    if profile:
        import cProfile
        import pstats
        import io

        # Create profiler
        profiler = cProfile.Profile()
        profiler.enable()

        try:
            result = func(*args, **kwargs)
            profiler.disable()

            end_time = time.time()
            duration = end_time - start_time

            # Get profile stats
            stats_buffer = io.StringIO()
            ps = pstats.Stats(profiler, stream=stats_buffer)
            ps.sort_stats("cumulative")
            ps.print_stats(20)  # Top 20 functions
            profile_output = stats_buffer.getvalue()

            print(f"✓ {operation_name} completed in {duration:.2f} seconds")
            print("\n" + "=" * 60)
            print(f"PROFILE RESULTS FOR {operation_name.upper()}")
            print("=" * 60)
            print(profile_output)
            print("=" * 60 + "\n")

            return result, duration, profile_output

        except Exception as e:
            profiler.disable()
            end_time = time.time()
            duration = end_time - start_time
            print(f"✗ {operation_name} failed after {duration:.2f} seconds: {e}")
            raise
    else:
        # Standard timing without profiling
        try:
            result = func(*args, **kwargs)
            end_time = time.time()
            duration = end_time - start_time
            print(f"✓ {operation_name} completed in {duration:.2f} seconds")
            return result, duration
        except Exception as e:
            end_time = time.time()
            duration = end_time - start_time
            print(f"✗ {operation_name} failed after {duration:.2f} seconds: {e}")
            raise


# Results storage
performance_results = {}
profile_results = {}  # Store profiling results separately
print("Performance monitoring utilities ready (now with profiling support)")

Performance monitoring utilities ready (now with profiling support)


In [12]:
rams_data.ds

<xarray.Dataset> Size: 85GB
Dimensions:     (x: 750, y: 440, time: 37, p: 2, z: 108)
Coordinates:
  * x           (x) float64 6kB -1.872e+05 -1.868e+05 ... 1.868e+05 1.872e+05
  * y           (y) float64 4kB -1.097e+05 -1.092e+05 ... 1.093e+05 1.098e+05
  * z           (z) float64 864B -11.63 12.2 38.42 ... 2.258e+04 2.283e+04
  * time        (time) datetime64[ns] 296B 1991-04-26T21:00:00 ... 1991-04-27
Dimensions without coordinates: p
Data variables: (12/22)
    GLAT        (x, y, time) float32 49MB dask.array<chunksize=(750, 440, 1), meta=np.ndarray>
    GLON        (x, y, time) float32 49MB dask.array<chunksize=(750, 440, 1), meta=np.ndarray>
    PATCH_AREA  (x, y, time, p) float32 98MB dask.array<chunksize=(750, 440, 1, 2), meta=np.ndarray>
    PCPRH       (x, y, time) float32 49MB dask.array<chunksize=(750, 440, 1), meta=np.ndarray>
    PCPRR       (x, y, time) float32 49MB dask.array<chunksize=(750, 440, 1), meta=np.ndarray>
    PCPVH       (x, y, z, time) float32 5GB dask.array<chunksize=(750, 440, 108, 1), meta=np.ndarray>
    ...          ...
    RV          (x, y, z, time) float32 5GB dask.array<chunksize=(750, 440, 108, 1), meta=np.ndarray>
    THETA       (x, y, z, time) float32 5GB dask.array<chunksize=(750, 440, 108, 1), meta=np.ndarray>
    TOPT        (x, y, time) float32 49MB dask.array<chunksize=(750, 440, 1), meta=np.ndarray>
    UC          (x, y, z, time) float32 5GB dask.array<chunksize=(750, 440, 108, 1), meta=np.ndarray>
    VC          (x, y, z, time) float32 5GB dask.array<chunksize=(750, 440, 108, 1), meta=np.ndarray>
    WC          (x, y, z, time) float32 5GB dask.array<chunksize=(750, 440, 108, 1), meta=np.ndarray>

In [ ]:
meshes = plot_rams_and_trajectories(
    pv_config=PVConfig(
        animation=False, show=True, interactive=False  # Don't show for profiling
    ),
    rams_data=None,  # No RAMS data
    trajectory_data=PVTrajectoryData(
        trajectory_ds=limited_trajectory_ds,
        varspecs=(trajectory_specs[0],),  # Just height-colored trajectories
    ),
)

2025-09-05 13:25:09.290 (  92.058s) [    7FCA9969E740]vtkXOpenGLRenderWindow.:1416  WARN| bad X server connection. DISPLAY=:99.0


Widget(value='<iframe src="http://localhost:41393/index.html?ui=P_0x7fc8201efe90_0&reconnect=auto" class="pyvi…

In [13]:
meshes = plot_rams_and_trajectories(
    pv_config=PVConfig(
        animation=False, show=True, interactive=False  # Don't show for profiling
    ),
    rams_data=PVRamsData(simulation_ds=rams_ds, varspecs=tuple(rams_specs)),
    trajectory_data=PVTrajectoryData(
        trajectory_ds=limited_trajectory_ds,
        varspecs=(trajectory_specs[0],),  # Just height-colored trajectories
    ),
)

Widget(value='<iframe src="http://localhost:41393/index.html?ui=P_0x7fc822452570_1&reconnect=auto" class="pyvi…

## Test 1: Basic Still Image Visualization

Test basic static visualization with trajectory data.

In [ ]:
# Profiling Test: Compare performance with and without RAMS data
print("=== Profiling Test: RAMS Data Performance Analysis ===")

# Test 1: Trajectory-only (baseline)
print("\n--- Baseline: Trajectory Only ---")
config_baseline = PVConfig(
    animation=False, show=False, interactive=False  # Don't show for profiling
)

basic_trajectory_data = PVTrajectoryData(
    trajectory_ds=limited_trajectory_ds,
    varspecs=(trajectory_specs[0],),  # Just height-colored trajectories
)


def run_trajectory_only():
    meshes = plot_rams_and_trajectories(
        pv_config=config_baseline,
        rams_data=None,  # No RAMS data
        trajectory_data=basic_trajectory_data,
    )
    return meshes


# Profile trajectory-only
try:
    meshes_baseline, duration_baseline, profile_baseline = time_operation(
        "Trajectory-only (baseline)", run_trajectory_only, profile=True
    )
    performance_results["trajectory_only"] = duration_baseline
    profile_results["trajectory_only"] = profile_baseline
except Exception as e:
    print(f"Baseline test failed: {e}")

# Test 2: RAMS + Trajectories (full test)
print("\n--- Full Test: RAMS + Trajectories ---")
config_full = PVConfig(
    animation=False, show=False, interactive=False  # Don't show for profiling
)


def run_rams_and_trajectories():
    meshes = plot_rams_and_trajectories(
        pv_config=config_full,
        rams_data=rams_data,  # Include RAMS data
        trajectory_data=basic_trajectory_data,
    )
    return meshes


# Profile full test with RAMS data
try:
    meshes_full, duration_full, profile_full = time_operation(
        "RAMS + Trajectories (full)", run_rams_and_trajectories, profile=True
    )
    performance_results["rams_and_trajectories"] = duration_full
    profile_results["rams_and_trajectories"] = profile_full

    # Calculate overhead
    rams_overhead = duration_full - duration_baseline
    overhead_ratio = (
        duration_full / duration_baseline if duration_baseline > 0 else float("inf")
    )

    print(f"\n📊 PERFORMANCE COMPARISON:")
    print(f"  Trajectory-only:     {duration_baseline:.2f}s")
    print(f"  RAMS + Trajectories: {duration_full:.2f}s")
    print(f"  RAMS Overhead:       {rams_overhead:.2f}s ({overhead_ratio:.1f}x slower)")

except Exception as e:
    print(f"Full test failed: {e}")
    import traceback

    traceback.print_exc()

# Test 3: RAMS-only (isolate RAMS performance)
print("\n--- RAMS Only Test ---")
config_rams_only = PVConfig(animation=False, show=False, interactive=False)


def run_rams_only():
    meshes = plot_rams_and_trajectories(
        pv_config=config_rams_only,
        rams_data=rams_data,  # Only RAMS data
        trajectory_data=None,  # No trajectories
    )
    return meshes


try:
    meshes_rams, duration_rams, profile_rams = time_operation(
        "RAMS-only", run_rams_only, profile=True
    )
    performance_results["rams_only"] = duration_rams
    profile_results["rams_only"] = profile_rams

    print(f"\n📊 RAMS-ONLY PERFORMANCE:")
    print(f"  RAMS-only:           {duration_rams:.2f}s")

except Exception as e:
    print(f"RAMS-only test failed: {e}")

print("\n=== Profiling Complete ===")
print("Check the profile outputs above to identify specific bottlenecks.")

=== Profiling Test: RAMS Data Performance Analysis ===

--- Baseline: Trajectory Only ---
Starting Trajectory-only (baseline)...


2025-09-05 11:14:01.195 ( 425.606s) [    7FA90B225740]vtkXOpenGLRenderWindow.:1416  WARN| bad X server connection. DISPLAY=:99.0


✓ Trajectory-only (baseline) completed in 11.13 seconds

PROFILE RESULTS FOR TRAJECTORY-ONLY (BASELINE)
         224338 function calls (200025 primitive calls) in 11.137 seconds

   Ordered by: cumulative time
   List reduced from 1416 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000   11.134   11.134 /tmp/ipykernel_1118401/3797366101.py:17(run_trajectory_only)
        1    0.000    0.000   11.015   11.015 /home/cmdavis4/projects/common_experimental/pvplotting/core_pvplotting.py:478(plot_rams_and_trajectories)
        1    0.000    0.000   11.015   11.015 /home/cmdavis4/projects/common_experimental/pvplotting/core_pvplotting.py:352(_process_time_point)
        1    0.000    0.000   10.976   10.976 /home/cmdavis4/projects/common_experimental/pvplotting/core_pvplotting.py:154(_create_meshes_for_frame)
        1    0.012    0.012   10.879   10.879 /home/cmdavis4/projects/common_experimental/pvplotting/trajec

## Profiling Test: RAMS Data Performance

This test specifically profiles the plotting performance with RAMS data to identify bottlenecks.

In [14]:
# Test 1: Basic still image with trajectories
print("=== Test 1: Basic Still Image Visualization ===")

# Create basic configuration
config_still = PVConfig(
    animation=False,
    show=True,
    interactive=False,
    # screenshot_path="./test_still_trajectories.png",
)

# Use only the first trajectory spec for basic test
basic_trajectory_data = PVTrajectoryData(
    trajectory_ds=limited_trajectory_ds,
    varspecs=(trajectory_specs[0],),  # Just height-colored trajectories
)


def run_still_plot():
    meshes = plot_rams_and_trajectories(
        pv_config=config_still,
        rams_data=rams_data,
        trajectory_data=basic_trajectory_data,
    )
    return meshes


try:
    meshes_still, duration_still = time_operation(
        "Still image visualization", run_still_plot
    )
    performance_results["still_image"] = duration_still

    print(f"Generated {len(meshes_still)} mesh groups")
    for time_key, mesh_list in meshes_still.items():
        print(f"  Time {time_key}: {len(mesh_list)} meshes")

except Exception as e:
    print(f"Still image test failed: {e}")
    import traceback

    traceback.print_exc()

=== Test 1: Basic Still Image Visualization ===
Starting Still image visualization...


Widget(value='<iframe src="http://localhost:41393/index.html?ui=P_0x7fc822402cf0_2&reconnect=auto" class="pyvi…

✓ Still image visualization completed in 4.30 seconds
Generated 1 mesh groups
  Time 1991-04-27T00:00:00.000000000: 2 meshes


## Test 2: Animation with Time Series

Test animation capabilities using the temporal data.

In [19]:
# Test 2: Animation (short duration for testing)
print("=== Test 2: Animation Visualization ===")

# Create animation configuration
config_anim = PVConfig(
    animation=True,
    gif_path="./test_animation_trajectories.gif",
    fps=5,  # Lower FPS for faster generation
    show=True,
    interactive=False,
    gif_scrubber=True,
)

# Use subset of time for faster animation (every 10th time step)
anim_trajectory_ds = limited_trajectory_ds.isel(time=slice(0, None, 10))
anim_trajectory_data = PVTrajectoryData(
    trajectory_ds=anim_trajectory_ds,
    varspecs=(trajectory_specs[0],),  # Height-colored trajectories
)

print(f"Animation will use {len(anim_trajectory_ds.time)} time steps")


def run_animation():
    meshes = plot_rams_and_trajectories(
        pv_config=config_anim,
        rams_data=None,  # Skip RAMS for animation test
        trajectory_data=anim_trajectory_data,
    )
    return meshes


try:
    meshes_anim, duration_anim = time_operation(
        "Animation visualization", run_animation
    )
    performance_results["animation"] = duration_anim

    print(f"Generated animation with {len(meshes_anim)} time frames")

    # Check if GIF was created
    gif_path = Path(config_anim.gif_path)
    if gif_path.exists():
        print(
            f"✓ Animation saved to {gif_path} ({gif_path.stat().st_size / 1024:.1f} KB)"
        )

except Exception as e:
    print(f"Animation test failed: {e}")
    import traceback

    traceback.print_exc()

=== Test 2: Animation Visualization ===
Animation will use 109 time steps
Starting Animation visualization...


Creating meshes by frame:   0%|          | 0/107 [00:00<?, ?it/s]

Widget(value='<iframe src="http://localhost:38369/index.html?ui=P_0x7fa64c3a09e0_5&reconnect=auto" class="pyvi…

✗ Animation visualization failed after 591.78 seconds: cannot access local variable 'gif_path' where it is not associated with a value
Animation test failed: cannot access local variable 'gif_path' where it is not associated with a value


Traceback (most recent call last):
  File "/tmp/ipykernel_1118401/4218595181.py", line 34, in <module>
    meshes_anim, duration_anim = time_operation(
                                 ^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_1118401/3548602369.py", line 59, in time_operation
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_1118401/4218595181.py", line 25, in run_animation
    meshes = plot_rams_and_trajectories(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cmdavis4/projects/common_experimental/pvplotting/core_pvplotting.py", line 636, in plot_rams_and_trajectories
    display(gif_scrubber(gif_path))
                         ^^^^^^^^
UnboundLocalError: cannot access local variable 'gif_path' where it is not associated with a value


## Test 3: Interactive Visualization

Test interactive PyVista features with time slider functionality.

In [ ]:
# Test 3: Interactive visualization with time slider
print("=== Test 3: Interactive Visualization ===")

# Create interactive configuration
config_interactive = PVConfig(
    animation=True,  # Animation=True with interactive=True creates slider
    interactive=True,
    show=True,
    fps=10,
)

# Use moderate time resolution for interactive exploration
interactive_trajectory_ds = limited_trajectory_ds.isel(time=slice(0, None, 5))
interactive_trajectory_data = PVTrajectoryData(
    trajectory_ds=interactive_trajectory_ds, varspecs=(trajectory_specs[0],)
)

print(f"Interactive plot will have {len(interactive_trajectory_ds.time)} time steps")


def run_interactive():
    meshes = plot_rams_and_trajectories(
        pv_config=config_interactive,
        rams_data=None,
        trajectory_data=interactive_trajectory_data,
    )
    return meshes


try:
    meshes_interactive, duration_interactive = time_operation(
        "Interactive visualization", run_interactive
    )
    performance_results["interactive"] = duration_interactive

    print(f"✓ Interactive visualization created with time slider")
    print("Use the slider at the bottom to explore different time steps")

except Exception as e:
    print(f"Interactive test failed: {e}")
    import traceback

    traceback.print_exc()

## Test 4: Multiple Trajectory Visualization Types

Test different trajectory visualization modes and styling options.

In [ ]:
# Test 4: Multiple trajectory types
print("=== Test 4: Multiple Trajectory Visualization Types ===")

# Test each trajectory spec individually
for i, traj_spec in enumerate(trajectory_specs):
    print(f"\nTesting trajectory spec {i+1}: {traj_spec.varname}")

    config_multi = PVConfig(
        animation=False,
        show=True,
        interactive=False,
        screenshot_path=f"./test_trajectory_type_{i+1}.png",
    )

    multi_trajectory_data = PVTrajectoryData(
        trajectory_ds=limited_trajectory_ds, varspecs=(traj_spec,)
    )

    def run_multi_test():
        meshes = plot_rams_and_trajectories(
            pv_config=config_multi,
            rams_data=None,
            trajectory_data=multi_trajectory_data,
        )
        return meshes

    try:
        meshes_multi, duration_multi = time_operation(
            f"Trajectory type {i+1} ({traj_spec.varname})", run_multi_test
        )
        performance_results[f"trajectory_type_{i+1}"] = duration_multi

        print(f"  ✓ Generated visualization for {traj_spec.varname}")
        print(f"    Scalar: {traj_spec.scalar}")
        print(f"    Particles: {traj_spec.particles}")
        print(f"    Colormap: {getattr(traj_spec, 'cmap', 'default')}")

    except Exception as e:
        print(f"  ✗ Failed trajectory type {i+1}: {e}")

## Test 5: Export and File Output Testing

In [ ]:
# Test 5: Export functionality
print("=== Test 5: Export and File Output Testing ===")

# Test various export formats
export_config = PVConfig(
    animation=False,
    show=False,  # Don't show, just export
    screenshot_path="./test_export_high_res.png",
    export_html=True,  # Also export HTML version
)


def run_export_test():
    meshes = plot_rams_and_trajectories(
        pv_config=export_config, rams_data=None, trajectory_data=basic_trajectory_data
    )
    return meshes


try:
    meshes_export, duration_export = time_operation(
        "Export functionality", run_export_test
    )
    performance_results["export"] = duration_export

    # Check generated files
    png_path = Path(export_config.screenshot_path)
    html_path = png_path.with_suffix(".html")

    if png_path.exists():
        print(f"✓ PNG export: {png_path} ({png_path.stat().st_size / 1024:.1f} KB)")

    if html_path.exists():
        print(f"✓ HTML export: {html_path} ({html_path.stat().st_size / 1024:.1f} KB)")

except Exception as e:
    print(f"Export test failed: {e}")
    import traceback

    traceback.print_exc()

## Test 6: Trajectory Group Analysis

Test visualization of different trajectory groups if group data is available.

In [ ]:
# Test 6: Trajectory groups (if available)
print("=== Test 6: Trajectory Group Analysis ===")

if "group" in trajectory_ds.data_vars:
    print("Group data available - testing group-based visualization")

    # Get unique groups
    unique_groups = np.unique(limited_trajectory_ds["group"].values)
    print(f"Available groups: {unique_groups}")

    # Test visualization of individual groups
    for group_name in unique_groups[:3]:  # Test first 3 groups
        if group_name == "":  # Skip empty group names
            continue

        print(f"\nTesting group: '{group_name}'")

        # Filter trajectories for this group
        group_mask = limited_trajectory_ds["group"] == group_name
        group_ds = limited_trajectory_ds.where(group_mask, drop=True)

        if len(group_ds.parcel_ix) == 0:
            print(f"  No parcels in group '{group_name}'")
            continue

        print(f"  Group '{group_name}' has {len(group_ds.parcel_ix)} parcels")

        # Create group-specific visualization
        group_config = PVConfig(
            animation=False,
            show=True,
            screenshot_path=f"./test_group_{group_name.replace('|', '_')}.png",
        )

        group_trajectory_data = PVTrajectoryData(
            trajectory_ds=group_ds, varspecs=(trajectory_specs[0],)  # Height-colored
        )

        def run_group_test():
            meshes = plot_rams_and_trajectories(
                pv_config=group_config,
                rams_data=None,
                trajectory_data=group_trajectory_data,
            )
            return meshes

        try:
            meshes_group, duration_group = time_operation(
                f"Group '{group_name}' visualization", run_group_test
            )
            performance_results[f"group_{group_name}"] = duration_group
            print(f"  ✓ Successfully visualized group '{group_name}'")

        except Exception as e:
            print(f"  ✗ Group '{group_name}' failed: {e}")

else:
    print("No group data available in trajectory dataset")
    print("Skipping group-based analysis")

In [ ]:
# Detailed profiling analysis
print("=== Detailed Profiling Analysis ===")

if profile_results:
    print("\nProfile Results Summary:")
    print("-" * 80)

    for test_name, profile_output in profile_results.items():
        print(f"\n🔍 {test_name.upper()}:")
        print("-" * 40)

        # Extract key performance metrics from profile output
        lines = profile_output.split("\n")

        # Find the header line and data lines
        data_started = False
        function_times = []

        for line in lines[5:25]:  # Skip header, take first 20 function calls
            if "ncalls" in line and "tottime" in line:
                data_started = True
                continue
            if data_started and line.strip():
                parts = line.split()
                if len(parts) >= 6:
                    try:
                        ncalls = parts[0]
                        tottime = float(parts[1])
                        cumtime = float(parts[3])
                        filename_func = " ".join(parts[5:])
                        function_times.append((cumtime, tottime, ncalls, filename_func))
                    except (ValueError, IndexError):
                        continue

        # Sort by cumulative time and show top functions
        function_times.sort(reverse=True, key=lambda x: x[0])

        print("Top time-consuming functions:")
        print(f"{'CumTime':<8} {'TotTime':<8} {'NCalls':<10} {'Function'}")
        print("-" * 60)

        for cumtime, tottime, ncalls, func_name in function_times[:10]:
            # Truncate long function names
            func_display = func_name[:45] + "..." if len(func_name) > 45 else func_name
            print(f"{cumtime:<8.3f} {tottime:<8.3f} {ncalls:<10} {func_display}")

        # Identify potential bottlenecks
        if function_times:
            top_func_time = function_times[0][0] if function_times else 0
            total_time = performance_results.get(test_name, 0)

            print(f"\n📈 Performance Analysis:")
            print(f"  Total execution time: {total_time:.2f}s")
            print(
                f"  Top function time:    {top_func_time:.2f}s"
                f" ({top_func_time/total_time*100 if total_time > 0 else 0:.1f}%)"
            )

            # Look for specific bottlenecks
            mesh_creation_time = sum(
                t[0]
                for t in function_times
                if "mesh" in t[3].lower() or "contour" in t[3].lower()
            )
            array_ops_time = sum(
                t[0]
                for t in function_times
                if "array" in t[3].lower() or "ravel" in t[3].lower()
            )
            pyvista_time = sum(
                t[0]
                for t in function_times
                if "pyvista" in t[3].lower() or "vtk" in t[3].lower()
            )

            if mesh_creation_time > 0.1:
                print(f"  Mesh creation time:   {mesh_creation_time:.2f}s")
            if array_ops_time > 0.1:
                print(f"  Array operations:     {array_ops_time:.2f}s")
            if pyvista_time > 0.1:
                print(f"  PyVista operations:   {pyvista_time:.2f}s")

else:
    print("No profiling results available. Run the profiling test first.")

# Compare performance across tests
if len(performance_results) >= 2:
    print("\n" + "=" * 80)
    print("PERFORMANCE COMPARISON SUMMARY")
    print("=" * 80)

    # Get specific comparisons
    traj_only = performance_results.get("trajectory_only", 0)
    rams_only = performance_results.get("rams_only", 0)
    full_test = performance_results.get("rams_and_trajectories", 0)

    if traj_only > 0 and full_test > 0:
        overhead = full_test - traj_only
        print(f"📊 RAMS Data Performance Impact:")
        print(f"   Trajectory baseline: {traj_only:.2f}s")
        print(f"   RAMS + Trajectories: {full_test:.2f}s")
        print(
            f"   RAMS overhead:       {overhead:.2f}s ({overhead/traj_only:.1f}x"
            " slower)"
        )

        if overhead > 5:
            print("⚠️  PERFORMANCE WARNING: RAMS data adds significant overhead")
            print("   Recommendations:")
            print("   - Check for inefficient mesh generation")
            print("   - Consider reducing contour resolution")
            print("   - Profile individual RAMS variable types")
        elif overhead > 2:
            print("⚠️  Moderate RAMS overhead detected")
        else:
            print("✅ RAMS overhead is acceptable")

    if rams_only > 0:
        print(f"   RAMS-only baseline:  {rams_only:.2f}s")

print("\n" + "=" * 80)

## Detailed Profiling Analysis

Analyze the profiling results to identify specific performance bottlenecks in RAMS data processing.

## Performance Analysis and Summary

In [ ]:
# Performance summary
print("=== Performance Analysis Summary ===")
print("\nTiming Results:")
print("-" * 50)

for test_name, duration in performance_results.items():
    print(f"{test_name:.<30} {duration:>8.2f}s")

if performance_results:
    total_time = sum(performance_results.values())
    avg_time = total_time / len(performance_results)
    print(f"{'Total time':.<30} {total_time:>8.2f}s")
    print(f"{'Average time':.<30} {avg_time:>8.2f}s")

# Data statistics
print("\nData Statistics:")
print("-" * 50)
print(f"Trajectory parcels: {len(limited_trajectory_ds.parcel_ix)}")
print(f"Time steps: {len(limited_trajectory_ds.time)}")
print(
    "Total trajectory points:"
    f" {len(limited_trajectory_ds.parcel_ix) * len(limited_trajectory_ds.time)}"
)
print(f"Trajectory variables: {len(list(limited_trajectory_ds.data_vars))}")

# Check output files
print("\nGenerated Files:")
print("-" * 50)
output_files = [
    "test_still_trajectories.png",
    "test_animation_trajectories.gif",
    "test_export_high_res.png",
    "test_export_high_res.html",
]

for filename in output_files:
    path = Path(filename)
    if path.exists():
        size_kb = path.stat().st_size / 1024
        print(f"✓ {filename} ({size_kb:.1f} KB)")
    else:
        print(f"✗ {filename} (not found)")

## Quality Assessment and Visual Validation

In [ ]:
# Visual validation checks
print("=== Visual Validation and Quality Assessment ===")

# Trajectory bounds checking
print("\nTrajectory Data Validation:")
print("-" * 30)

x_min, x_max = float(limited_trajectory_ds.x.min()), float(
    limited_trajectory_ds.x.max()
)
y_min, y_max = float(limited_trajectory_ds.y.min()), float(
    limited_trajectory_ds.y.max()
)
z_min, z_max = float(limited_trajectory_ds.z.min()), float(
    limited_trajectory_ds.z.max()
)

print(f"X range: {x_min:.0f} to {x_max:.0f} m ({(x_max-x_min)/1000:.1f} km span)")
print(f"Y range: {y_min:.0f} to {y_max:.0f} m ({(y_max-y_min)/1000:.1f} km span)")
print(f"Z range: {z_min:.0f} to {z_max:.0f} m ({(z_max-z_min)/1000:.1f} km span)")

# Check for reasonable atmospheric values
print("\nPhysical Plausibility Checks:")
print("-" * 30)
if z_min >= 0 and z_max <= 20000:  # Reasonable tropospheric heights
    print("✓ Trajectory heights are physically reasonable (0-20km)")
else:
    print(f"⚠ Trajectory heights may be unusual: {z_min:.0f} to {z_max:.0f} m")

# Check trajectory continuity
sample_trajectory = limited_trajectory_ds.isel(parcel_ix=0)
x_diff = np.diff(sample_trajectory.x.values)
y_diff = np.diff(sample_trajectory.y.values)
z_diff = np.diff(sample_trajectory.z.values)
displacement = np.sqrt(x_diff**2 + y_diff**2 + z_diff**2)

max_displacement = np.nanmax(displacement)
avg_displacement = np.nanmean(displacement)

print(f"Sample trajectory displacement per timestep:")
print(f"  Average: {avg_displacement:.1f} m")
print(f"  Maximum: {max_displacement:.1f} m")

if max_displacement < 10000:  # Less than 10 km per timestep seems reasonable
    print("✓ Trajectory displacements appear reasonable")
else:
    print("⚠ Large trajectory displacements detected - check timestep")

# Performance assessment
print("\nPerformance Assessment:")
print("-" * 30)
if "still_image" in performance_results:
    still_time = performance_results["still_image"]
    if still_time < 10:
        print(f"✓ Still image generation is fast ({still_time:.1f}s)")
    elif still_time < 30:
        print(f"~ Still image generation is acceptable ({still_time:.1f}s)")
    else:
        print(f"⚠ Still image generation is slow ({still_time:.1f}s)")

if "animation" in performance_results:
    anim_time = performance_results["animation"]
    n_frames = len(anim_trajectory_ds.time) if "anim_trajectory_ds" in locals() else 1
    time_per_frame = anim_time / max(n_frames, 1)

    print(
        f"Animation: {anim_time:.1f}s for {n_frames} frames"
        f" ({time_per_frame:.2f}s/frame)"
    )
    if time_per_frame < 2:
        print("✓ Animation frame generation is efficient")
    else:
        print("⚠ Animation frame generation may be slow")

print("\n=== Testing Complete ===")
print("Review the generated visualizations to assess visual quality and correctness.")

## Conclusions and Recommendations

This notebook has tested the core functionality of the refactored pvplotting library with real atmospheric simulation data. Key findings:

### Functionality Tested:
1. **Data Loading**: Successfully converted external data to new `PVTrajectoryData` format
2. **Still Images**: Basic trajectory visualization with height-based coloring
3. **Animations**: Time-series animations with GIF export
4. **Interactive Views**: PyVista-based interactive exploration with time sliders
5. **Multiple Styles**: Different trajectory visualization modes (lines, particles, groups)
6. **Export Options**: PNG screenshots and HTML export functionality

### Performance Monitoring:
- Timing data collected for all major operations
- Memory usage and file size tracking
- Scalability assessment with real data volumes

### Quality Validation:
- Physical plausibility checks on trajectory data
- Spatial and temporal continuity validation
- Visual output verification

This example notebook can serve as both a comprehensive test suite and a practical guide for using the refactored plotting library with real scientific data.